In [ ]:
%load_ext autoreload
%reload_ext autoreload

%autoreload 2

import mlflow as mlf
import sys
import os
sys.path.append('../')

from model.utils import load_model, load_dataset
from model.train import train_model

## Smoke tests for rebertuito model

In [ ]:
ds_path = os.path.abspath(os.path.join('..', 'model', 'tweets_parsed.csv'))

ds = load_dataset(ds_path, force=False)
for key in ds.keys():
  for v in ds[key]:
    print(key, v)
    break

model, tokenizer = load_model(base_model='pysentimiento/robertuito-base-uncased')

## Smoke test for ReBERTuito+BLSTM

In [ ]:
ds_path = os.path.abspath(os.path.join('..', 'model', 'tweets_parsed.csv'))

ds = load_dataset(ds_path, force=False)
for key in ds.keys():
  for v in ds[key]:
    print(key, v)
    break

model, tokenizer = load_model(base_model='pysentimiento/robertuito-base-uncased', blstm=True)

Smoke test train

In [ ]:
train_model(limit=8)

## Analysis runs

In [ ]:
import mlflow as mlf
from transformers import (
    AutoModelForSequenceClassification,
)

runs_path = os.path.abspath(os.path.join('..', 'model', 'mlruns'))

print(runs_path)

mlf.set_tracking_uri('file:/ ' + runs_path)

columns = ['run_id', 'status', 'metrics.train_runtime', 'metrics.eval_macro_recall', 'metrics.eval_macro_precision', 'metrics.eval_macro_f1', 'metrics.train_loss', 'artifact_uri']

runs = mlf.search_runs(
  filter_string='status="FINISHED"',
  order_by=['metrics.eval_macro_f1 DESC']
)

runs[columns]

In [ ]:
#load model with best f1 score
best_run = runs.iloc[0]

best_run['run_id']

In [11]:
_model = AutoModelForSequenceClassification.from_pretrained(f'{best_run["artifact_uri"]}/model'.replace('file:///', ''))
_model